In [1]:
from vllm import LLM, SamplingParams
model_name = "/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f"
# model_name = "/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa"

llm = LLM(
            model=model_name,
            dtype="half"
)

sampling_params = SamplingParams(
            temperature=0.7,
            top_p=0.95,
            max_tokens=500
)

/home/anamikaghosh_umass_edu/.conda/envs/gpu-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-02 21:57:13,353	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 04-02 21:57:15 __init__.py:207] Automatically detected platform cuda.
WARNING 04-02 21:57:15 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 04-02 21:57:30 config.py:549] This model supports multiple tasks: {'score', 'generate', 'classify', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 04-02 21:57:30 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f', speculative_config=None, tokenizer='/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quanti

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.34s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.61it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.37it/s]



INFO 04-02 21:57:34 model_runner.py:1115] Loading model weights took 4.7199 GB
INFO 04-02 21:57:35 worker.py:267] Memory profiling takes 0.49 seconds
INFO 04-02 21:57:35 worker.py:267] the current vLLM instance can use total_gpu_memory (44.52GiB) x gpu_memory_utilization (0.90) = 40.07GiB
INFO 04-02 21:57:35 worker.py:267] model weights take 4.72GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 34.80GiB.
INFO 04-02 21:57:35 executor_base.py:111] # cuda blocks: 28506, # CPU blocks: 3276
INFO 04-02 21:57:35 executor_base.py:116] Maximum concurrency for 4096 tokens per request: 111.35x
INFO 04-02 21:57:36 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:11<00:00,  2.94it/s]

INFO 04-02 21:57:48 model_runner.py:1562] Graph capturing finished in 12 secs, took 0.31 GiB
INFO 04-02 21:57:48 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 14.44 seconds


In [2]:
query = "sort a list in python"
prompts = [f"Expand the following search query with additional relevant terms and make 20 expanded queries. Seperate each expanded query with '\n' seperator. \nQuery: {query}\nExpanded Queries:"]

results = llm.generate(prompts, sampling_params)
explanations = [result.outputs[0].text for result in results]

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.60s/it, est. speed input: 16.56 toks/s, output: 110.90 toks/s]


In [3]:
explanations

['\n1. sort a list in python using built-in sort function\n2. sort a list in python using sorted() function\n3. sort a list in python using lambda function\n4. sort a list in python using custom sort function\n5. sort a list in python using key function\n6. sort a list in python using reverse order\n7. sort a list in python using multiple keys\n8. sort a list in python using custom key function\n9. sort a list in python using custom comparison function\n10. sort a list in python using custom sorting algorithm\n11. sort a list in python using heapq module\n12. sort a list in python using bisect module\n13. sort a list in python using heapq.nsmallest() function\n14. sort a list in python using heapq.nlargest() function\n15. sort a list in python using sorted() function with key and reverse\n16. sort a list in python using list.sort() method with key and reverse\n17. sort a list in python using sorted() function with custom comparison function\n18. sort a list in python using list.sort() 

In [ ]:
# Ensure you have installed the required packages:
# pip install rank-bm25 nltk

from rank_bm25 import BM25Okapi
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re

nltk.download('punkt')


splitted_explanations = [line.strip() for line in explanations[0].split('\n') if line.strip()]

splitted_explanations = [re.sub(r'^\d+\.\s*', '', exp) for exp in splitted_explanations]

original_query = "sort a list in python"

tokenized_explanations = [word_tokenize(exp.lower()) for exp in splitted_explanations]
tokenized_query = word_tokenize(original_query.lower())

bm25 = BM25Okapi(tokenized_explanations)

scores = bm25.get_scores(tokenized_query)
print("BM25 Scores for candidate explanations:", scores)

best_index = np.argmax(scores)
best_explanation = explanations[best_index]

print("\nOriginal Query:", original_query)
print("Best Explanation:", best_explanation)


BM25 Scores for candidate explanations: [1.18711262 1.047546   1.15146349 1.18711262 1.15146349 1.15146349
 1.15146349 1.09704935 1.09704935 1.09704935 1.15146349 1.15146349
 1.047546   1.047546   0.88737757 0.88737757 0.88737757 0.88737757
 1.047546   1.047546  ]

Original Query: sort a list in python
Best Explanation: 
1. sort a list in python using built-in sort function
2. sort a list in python using sorted() function
3. sort a list in python using lambda function
4. sort a list in python using custom sort function
5. sort a list in python using key function
6. sort a list in python using reverse order
7. sort a list in python using multiple keys
8. sort a list in python using custom key function
9. sort a list in python using custom comparison function
10. sort a list in python using custom sorting algorithm
11. sort a list in python using heapq module
12. sort a list in python using bisect module
13. sort a list in python using heapq.nsmallest() function
14. sort a list in python

[nltk_data] Downloading package punkt to
[nltk_data]     /home/anamikaghosh_umass_edu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import nltk
nltk.download('punkt')

query = "sort a list in python"

# Step 1: Prompt to generate 20 paraphrased/synonym-expanded versions of the query
prompt = f"""Expand the following search query using synonyms or by rephrasing, maintaining the original intent.
Query: {query}
Expanded Queries:
"""

# Call your LLM to get the list of expanded queries
results = llm.generate([prompt], sampling_params)
raw_output = results[0].outputs[0].text.strip()

# Step 2: Post-process LLM output
expanded_queries = [q.strip() for q in raw_output.split("\n") if q.strip()]

# Step 3: Concatenate all expanded queries into a single, enriched query string
rich_query = " ".join(expanded_queries)

print("🔍 Enriched Query for BM25:")
print(rich_query)


[nltk_data] Downloading package punkt to
[nltk_data]     /home/anamikaghosh_umass_edu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.50s/it, est. speed input: 7.34 toks/s, output: 111.24 toks/s]

🔍 Enriched Query for BM25:
1. Arrange a list in Python 2. Order a list in Python 3. Sort a collection of elements in Python 4. Rearrange a list in Python 5. Arrange elements in a list in Python 6. Order elements in a list in Python 7. Sort a sequence of items in Python 8. Reorder a sequence of items in Python 9. Sort a list of elements in Python 10. Order a list of elements in Python Here is a Python function that sorts a list: ```python def sort_list(input_list): return sorted(input_list) ``` This function uses the built-in `sorted()` function to sort the input list in ascending order. If you want to sort the list in descending order, you can use the `reverse=True` argument: ```python def sort_list_descending(input_list): return sorted(input_list, reverse=True) ``` You can also use the `sort()` method of the list object to sort the list in-place: ```python def sort_list_in_place(input_list): input_list.sort() return input_list ``` The `sort()` method sorts the list in-place, meaning t

In [14]:
rich_query

'1. Arrange a list in Python 2. Order a list in Python 3. Sort a collection of elements in Python 4. Rearrange a list in Python 5. Arrange elements in a list in Python 6. Order elements in a list in Python 7. Sort a sequence of items in Python 8. Reorder a sequence of items in Python 9. Sort a list of elements in Python 10. Order a list of elements in Python Here is a Python function that sorts a list: ```python def sort_list(input_list): return sorted(input_list) ``` This function uses the built-in `sorted()` function to sort the input list in ascending order. If you want to sort the list in descending order, you can use the `reverse=True` argument: ```python def sort_list_descending(input_list): return sorted(input_list, reverse=True) ``` You can also use the `sort()` method of the list object to sort the list in-place: ```python def sort_list_in_place(input_list): input_list.sort() return input_list ``` The `sort()` method sorts the list in-place, meaning that it modifies the origin

In [10]:
corpus_docs = [
    "Sort a list in Python using sorted(). def sort_list(lst): return sorted(lst)",
    "Sort list elements in-place using list.sort(). def in_place_sort(lst): lst.sort()",
    "Bubble sort implementation for list sorting. def bubble_sort(arr): for i in range(len(arr)): for j in range(0, len(arr)-i-1): if arr[j] > arr[j+1]: arr[j], arr[j+1] = arr[j+1], arr[j]",
    "Reverse sort a list in Python. def reverse_sort(lst): return sorted(lst, reverse=True)",
    "Recursive merge sort in Python. def merge_sort(arr): if len(arr) > 1: mid = len(arr)//2; L = arr[:mid]; R = arr[mid:]; ..."
]

In [11]:
tokenized_corpus = [nltk.word_tokenize(doc.lower()) for doc in corpus_docs]
bm25 = BM25Okapi(tokenized_corpus)


In [40]:
# Step 2: Tokenize the query
tokenized_query = nltk.word_tokenize(verbose_query.lower())

# Step 3: Score the corpus using BM25
scores = bm25.get_scores(tokenized_query)

# Step 4: Rank documents by BM25 score
ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
top_docs = [(corpus_docs[i], scores[i]) for i in ranked_indices]

# Step 5: Print top results
print("🔍 Top Retrieved Code Snippets:\n")
for i, (doc, score) in enumerate(top_docs[:5], 1):
    print(f"{i}. [Score: {score:.2f}]\n{doc}\n")

🔍 Top Retrieved Code Snippets:

1. [Score: 6.75]
Bubble sort implementation for list sorting. def bubble_sort(arr): for i in range(len(arr)): for j in range(0, len(arr)-i-1): if arr[j] > arr[j+1]: arr[j], arr[j+1] = arr[j+1], arr[j]

2. [Score: 3.54]
Reverse sort a list in Python. def reverse_sort(lst): return sorted(lst, reverse=True)

3. [Score: 2.26]
Sort a list in Python using sorted(). def sort_list(lst): return sorted(lst)

4. [Score: 0.88]
Recursive merge sort in Python. def merge_sort(arr): if len(arr) > 1: mid = len(arr)//2; L = arr[:mid]; R = arr[mid:]; ...

5. [Score: 0.60]
Sort list elements in-place using list.sort(). def in_place_sort(lst): lst.sort()



In [85]:
query = "sort a list in python"

prompt = f"""
Instruction: 
Respond with only 1 improved query. Make the following query slightly more detailed, while keeping its original meaning. Please do not provide any code, just a rephrased query. Say it in third-person perspective.

Query:{query}
Answer:
"""

# LLM call
results = llm.generate([prompt], sampling_params)
verbose_query = results[0].outputs[0].text.strip()

print("Verbose Query:\n", verbose_query)


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.68it/s, est. speed input: 232.69 toks/s, output: 92.33 toks/s]

Verbose Query:
 How to systematically arrange a sequence of elements in ascending or descending order using Python's built-in sorting functionality?


In [69]:
# import re

# raw_output = results[0].outputs[0].text.strip()

# match = re.search(r"Improved Query:\s*(.*)", raw_output, re.IGNORECASE)
# improved_query = match.group(1).strip() if match else raw_output.strip()

# print("Improved Query:", verbose_query)

In [102]:
corpus_docs = [
    "Sort a list in Python using sorted(). def sort_list(lst): return sorted(lst)",
    "Sort list elements in-place using list.sort(). def in_place_sort(lst): lst.sort()",
    "Bubble sort implementation for list sorting. def bubble_sort(arr): for i in range(len(arr)): for j in range(0, len(arr)-i-1): if arr[j] > arr[j+1]: arr[j], arr[j+1] = arr[j+1], arr[j]",
    "Reverse sort a list in Python. def reverse_sort(lst): return sorted(lst, reverse=True)",
    "properly arrange a sequence of elements in order,def reverse_sort(lst): return sorted(lst, reverse=True) ?",
    "Recursive merge sort in Python. def merge_sort(arr): if len(arr) > 1: mid = len(arr)//2; L = arr[:mid]; R = arr[mid:]; ..."
]

In [103]:
tokenized_corpus = [nltk.word_tokenize(doc.lower()) for doc in corpus_docs]
bm25 = BM25Okapi(tokenized_corpus)


In [104]:
# Step 2: Tokenize the query
tokenized_query = nltk.word_tokenize(verbose_query.lower())

# Step 3: Score the corpus using BM25
scores = bm25.get_scores(tokenized_query)

# Step 4: Rank documents by BM25 score
ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
top_docs = [(corpus_docs[i], scores[i]) for i in ranked_indices]

# Step 5: Print top results
print("🔍 Top Retrieved Code Snippets:\n")
for i, (doc, score) in enumerate(top_docs[:5], 1):
    print(f"{i}. [Score: {score:.2f}]\n{doc}\n")

🔍 Top Retrieved Code Snippets:

1. [Score: 8.33]
properly arrange a sequence of elements in order,def reverse_sort(lst): return sorted(lst, reverse=True) ?

2. [Score: 1.47]
Sort list elements in-place using list.sort(). def in_place_sort(lst): lst.sort()

3. [Score: 1.03]
Bubble sort implementation for list sorting. def bubble_sort(arr): for i in range(len(arr)): for j in range(0, len(arr)-i-1): if arr[j] > arr[j+1]: arr[j], arr[j+1] = arr[j+1], arr[j]

4. [Score: 0.87]
Sort a list in Python using sorted(). def sort_list(lst): return sorted(lst)

5. [Score: 0.18]
Reverse sort a list in Python. def reverse_sort(lst): return sorted(lst, reverse=True)



In [92]:
print(verbose_query)

How to systematically arrange a sequence of elements in ascending or descending order using Python's built-in sorting functionality?
